In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM,Dense, Dropout, BatchNormalization
from keras.preprocessing.sequence import TimeseriesGenerator

%matplotlib inline  




C:\Users\stesk\.conda\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\stesk\.conda\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\stesk\.conda\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\stesk\.conda\envs\PythonData\lib\site-packages\tensorflow\

In [2]:
df = yf.download('AMZN',start='2016-06-01', interval='1d',  end='2021-06-01',progress=False)

df.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)
df["Date"] = df.index
df.head()
 
#data.plot(figsize=(10,10))

,Close,Adj Close,Date
Date,,,
2016-06-01,719.440002,719.440002,2016-06-01
2016-06-02,728.239990,728.239990,2016-06-02
2016-06-03,725.539978,725.539978,2016-06-03
2016-06-06,726.729980,726.729980,2016-06-06
2016-06-07,723.739990,723.739990,2016-06-07


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1258 entries, 2016-06-01 to 2021-05-28
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Close      1258 non-null   float64       
 1   Adj Close  1258 non-null   float64       
 2   Date       1258 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2)
memory usage: 39.3 KB


In [4]:
close_data = df['Close'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['Date'][:split]
date_test = df['Date'][split:]

print(len(close_train))
print(len(close_test))

1006
252


In [5]:
look_back = 15

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [6]:
model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)


Epoch 1/25
50/50 [==============================] - 1s 13ms/step - loss: 2318777.9032
Epoch 2/25
50/50 [==============================] - 0s 5ms/step - loss: 2318640.1449
Epoch 3/25
50/50 [==============================] - 0s 5ms/step - loss: 2318502.7082
Epoch 4/25
50/50 [==============================] - 0s 5ms/step - loss: 2318365.2590
Epoch 5/25
50/50 [==============================] - 0s 5ms/step - loss: 2318227.8100
Epoch 6/25
50/50 [==============================] - 0s 5ms/step - loss: 2318091.1517
Epoch 7/25
50/50 [==============================] - 0s 5ms/step - loss: 2317954.0046
Epoch 8/25
50/50 [==============================] - 0s 5ms/step - loss: 2317816.3802
Epoch 9/25
50/50 [==============================] - 0s 5ms/step - loss: 2317679.3462
Epoch 10/25
50/50 [==============================] - 0s 5ms/step - loss: 2317541.7075
Epoch 11/25
50/50 [==============================] - 0s 5ms/step - loss: 2317405.1680
Epoch 12/25
50/50 [==============================] - 0s 5ms/s

In [7]:
#Test
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Amazon Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [10]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)
print(forecast_dates)

[Timestamp('2021-05-28 00:00:00', freq='D'), Timestamp('2021-05-29 00:00:00', freq='D'), Timestamp('2021-05-30 00:00:00', freq='D'), Timestamp('2021-05-31 00:00:00', freq='D'), Timestamp('2021-06-01 00:00:00', freq='D'), Timestamp('2021-06-02 00:00:00', freq='D'), Timestamp('2021-06-03 00:00:00', freq='D'), Timestamp('2021-06-04 00:00:00', freq='D'), Timestamp('2021-06-05 00:00:00', freq='D'), Timestamp('2021-06-06 00:00:00', freq='D'), Timestamp('2021-06-07 00:00:00', freq='D'), Timestamp('2021-06-08 00:00:00', freq='D'), Timestamp('2021-06-09 00:00:00', freq='D'), Timestamp('2021-06-10 00:00:00', freq='D'), Timestamp('2021-06-11 00:00:00', freq='D'), Timestamp('2021-06-12 00:00:00', freq='D'), Timestamp('2021-06-13 00:00:00', freq='D'), Timestamp('2021-06-14 00:00:00', freq='D'), Timestamp('2021-06-15 00:00:00', freq='D'), Timestamp('2021-06-16 00:00:00', freq='D'), Timestamp('2021-06-17 00:00:00', freq='D'), Timestamp('2021-06-18 00:00:00', freq='D'), Timestamp('2021-06-19 00:00:00'

In [9]:
trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)

layout = go.Layout(
    title = "Amazon Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()